In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import scipy
from scipy import stats
from sklearn.utils import shuffle
from matplotlib.lines import Line2D
import os
from ray import tune

print(os.cpu_count())

# workloads = ["readseq"]#, "readrandomwriterandom", "mixgraph", "updaterandom"] #["fileserver", "mixgraph", "oltp", "rwrandom", "readseq", "readwhilewriting", "varmail", "webserver"]

# accuracy as comparison of distribuitions
# comparitive analysis against table base[HP], cart model[CMU], decision tree,
# 
# -Plan-
# Send decision tree graph
# let's put latency in the buckets and predict the buckets
# - powers of 2
# - ?
# differnt hw's
    # mq-deadline ? setting all deadlines with predictions
    # cache the I/O?
    # accuracy is based not the class of buckets how far from correct bucket

############### FEATURES ##################
# 0. issue time to driver - inter-arrival time use it - time diff between two consecutive accesses
# 1. operation type
# 2. abs(LBA diffs)
# 3. time diff between completions
# 4. tag
# 5. size - length of the I/O operation
# 6. queue size
# 7. Latency

#-> Backtrack the decisions

2024-05-24 20:11:26,913	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-24 20:11:28,539	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


64


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


In [4]:
workloads = ["readseq0", "readseq1", "readseq2", "readseq3", "readwhilewriting0", "readwhilewriting1", "readreverse0", "readrandomwriterandom0", "readrandom0", "mixgraph0", "fillseq0", "fillseq1", "fill100k0", "fill100k1"]
config = {"lr" : 0.01, "momentum" : 0.90}
num_epoch = 50
do_binary_classification = False

In [5]:
experiments = {}
for w in workloads:
    # print("reading data for", w)
    data = np.loadtxt("data/" + w + ".csv",
                      dtype=np.double, skiprows=1, delimiter=",")
    experiments[w] = data

In [6]:
from collections import defaultdict
combined_data = defaultdict(list)
for workload, values in experiments.items():
    prefix = ''.join(filter(str.isalpha, workload))
    combined_data[prefix].extend(values)

In [7]:
max_length = max(len(values) for values in combined_data.values())
def duplicate_to_match_length(data, target_length):
    if len(data) == 0:
        return []
    repeated_data = (data * (target_length // len(data))) + data[:target_length % len(data)]
    return repeated_data

equalized_data = {key: duplicate_to_match_length(values, max_length) for key, values in combined_data.items()}

In [8]:
equalized_data.keys()

dict_keys(['readseq', 'readwhilewriting', 'readreverse', 'readrandomwriterandom', 'readrandom', 'mixgraph', 'fillseq', 'fillk'])

In [9]:
for idx, key in enumerate(equalized_data.keys()):
        equalized_data[key] = np.hstack((equalized_data[key], np.zeros((len(equalized_data[key]), 1))))
        equalized_data[key][:,-1] = idx

In [10]:
data_all = []
for values in equalized_data.values():
    data_all.extend(values)

In [11]:
data_all = np.array(data_all)

In [12]:
np.shape(data_all)

(1159888, 9)

In [13]:
type(data_all)

numpy.ndarray

In [14]:
# split data into input and output
input = data_all[:,0:-1]
output = data_all[:,-1]

In [15]:
input = np.concatenate((input[:, 0:3], input[:, 4:6]), axis=1)

In [16]:
# get normalized input (output stays unnormalized)
norm_input = stats.zscore(input, axis=0)

print(min(output), max(output))
output = output.reshape(-1, 1)

norm_input, output, input = shuffle(norm_input, output, input)

0.0 7.0


In [17]:
output = output.reshape(-1)

In [18]:
categories = np.unique(output)
bin_output = np.zeros((len(output), len(categories)))
for i, value in enumerate(output):
    index = np.where(categories == value)[0][0]
    bin_output[i, index] = 1

In [25]:
def np_out_csv(ar, name=None):
    if name == None:
        name = "file"
    import pandas as pd
    df = pd.DataFrame(ar)
    df.to_csv(index=False, header=False, sep=" ", path_or_buf=name + ".csv", float_format="%015.6f") #save to file

In [19]:
for i in range(len(input[0])):
    print(np.mean(input[:,i]))

print()

for i in range(len(input[0])):
    print(np.std(input[:,i]))

0.0058691436979165235
2.353178927620598
1619459.160498255
16.327928213758568
81.25199674451326

0.15204302763446695
2.241893034781976
4724708.014463893
4.708433207358461
219.24495606131507


In [20]:
print(norm_input.shape)
print(bin_output.shape)

(1159888, 5)
(1159888, 8)


In [21]:
from sklearn.model_selection import train_test_split
input_train, input_test, bin_output_train, bin_output_test = train_test_split(norm_input, bin_output, test_size=0.33, random_state=42)

In [22]:
class latencyPredictor(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(latencyPredictor, self).__init__()
        self.layer_1 = torch.nn.Linear(inputSize, 512)
        self.layer_2 = torch.nn.Linear(512, 128)
        self.layer_3 = torch.nn.Linear(128, 64)
        self.layer_out = torch.nn.Linear(64, outputSize)
        
        self.relu = torch.nn.Sigmoid()
        # self.softmax = torch.nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.layer_1(x)
        x = self.relu(x)
        
        x = self.layer_2(x)
        x = self.relu(x)
        
        x = self.layer_3(x)
        x = self.relu(x)
        
        x = self.layer_out(x)
        # added softmax to turn our results into preds
        # x = self.softmax(x)
        # Nevermind! We don't have access to a Softmax layer!
        return x

In [23]:
# use multiple gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [38]:
# state_dict2 = {}
# state_dict2["module.layer_1.weight"] = state_dict["module.module.layer_1.weight"]
# state_dict2["module.layer_2.weight"] = state_dict["module.module.layer_2.weight"]
# state_dict2["module.layer_3.weight"] = state_dict["module.module.layer_3.weight"]
# state_dict2["module.layer_out.weight"] = state_dict["module.module.layer_out.weight"]
# state_dict2["module.layer_1.bias"] = state_dict["module.module.layer_1.bias"]
# state_dict2["module.layer_2.bias"] = state_dict["module.module.layer_2.bias"]
# state_dict2["module.layer_3.bias"] = state_dict["module.module.layer_3.bias"]
# state_dict2["module.layer_out.bias"] = state_dict["module.module.layer_out.bias"]

In [24]:
class_size = 8

In [25]:
# do this if you want to load a previous model
latencyPred = latencyPredictor(norm_input.shape[1], int(class_size))
latencyPred.double()
latencyPred = torch.nn.DataParallel(latencyPred)
latencyPred.to(device)
state_dict = torch.load("saved/model.pt")
latencyPred.load_state_dict(state_dict)
torch.set_num_threads(os.cpu_count())
latencyPred.train()

DataParallel(
  (module): latencyPredictor(
    (layer_1): Linear(in_features=5, out_features=512, bias=True)
    (layer_2): Linear(in_features=512, out_features=128, bias=True)
    (layer_3): Linear(in_features=128, out_features=64, bias=True)
    (layer_out): Linear(in_features=64, out_features=8, bias=True)
    (relu): Sigmoid()
  )
)

In [26]:
latencyPred.eval()

DataParallel(
  (module): latencyPredictor(
    (layer_1): Linear(in_features=5, out_features=512, bias=True)
    (layer_2): Linear(in_features=512, out_features=128, bias=True)
    (layer_3): Linear(in_features=128, out_features=64, bias=True)
    (layer_out): Linear(in_features=64, out_features=8, bias=True)
    (relu): Sigmoid()
  )
)

In [ ]:
# latencyPred = latencyPredictor(input.shape[1], 1)
print(class_size)
if do_binary_classification:
    class_size = 2
latencyPred = latencyPredictor(norm_input.shape[1], int(class_size))
latencyPred.double()
torch.set_num_threads(os.cpu_count())

In [ ]:
# criterion = torch.nn.MSELoss()
print(device)
criterion = torch.nn.CrossEntropyLoss()
latencyPred = torch.nn.DataParallel(latencyPred)
optimizer = torch.optim.SGD(latencyPred.parameters(), lr=0.01, momentum=0.961)
latencyPred.to(device)

In [ ]:
from IPython.display import clear_output

def plot_loss(epochs, losses, tests, save):
    clear_output(wait=True)
    fig, ax1 = plt.subplots(figsize =(10, 4))
    ax2 = ax1.twinx()
    ax1.plot(epochs, losses, 'b-')
    ax2.plot(epochs, tests, 'g-')
    custom_lines = [Line2D([0], [0], color='b', lw=4),
                Line2D([0], [0], color='g', lw=4)]
    plt.legend(custom_lines, ['Loss', "Accuracy"])
    if save:
        plt.savefig("loss-acc.png", dpi=200)
    plt.show()

def plot_all(epochs, losses, tests, correct_dist_idx, labels):
    clear_output(wait=True)
    fig, axs = plt.subplots(4,1,figsize =(10, 16))
    ax2 = axs[0].twinx()
    axs[0].plot(epochs, losses, 'b-')
    ax2.plot(epochs, tests, 'g-')
    
    correct_dist = []
    incorrect_dist = []
    correct_ones = np.zeros(bin_output.shape[0], dtype = int)
    incorrect_ones = np.zeros(bin_output.shape[0], dtype = int)
    for idx, answer in enumerate(correct_dist_idx):
        if (answer.item()):
            correct_dist.append(int(labels[idx]))
            correct_ones[int(labels[idx])] = correct_ones[int(labels[idx])] + 1
        else:
            incorrect_dist.append(int(labels[idx]))
            incorrect_ones[int(labels[idx])] = incorrect_ones[int(labels[idx])] + 1
    
    correct_dist = np.array(correct_dist)
    incorrect_dist = np.array(incorrect_dist)

    print(len(correct_dist_idx), correct_dist.shape, incorrect_dist.shape)
    print(correct_ones)
    print(incorrect_ones)

    bin_list = range(int(class_size))
    axs[1].set_yscale("log")
    axs[1].hist(correct_dist, bin_list)
    axs[2].set_yscale("log")
    axs[2].hist(incorrect_dist, bin_list)
    axs[3].set_yscale("log")
    axs[3].hist(bin_output, bin_list)

    plt.show()

In [27]:
input_train = torch.DoubleTensor(input_train).to(device)
# labels = torch.DoubleTensor(bin_output)
bin_output_train = torch.LongTensor(bin_output_train).to(device)

input_test = torch.DoubleTensor(input_test).to(device)
bin_output_test = torch.LongTensor(bin_output_test).to(device)

In [ ]:
from matplotlib.pyplot import plot
import collections.abc

epochs = []
losses = []
tests = []

batch_size = 1000


for epoch in range(300): # 1000

    batch_test = []
    batch_loss = []

    for batch in range(0, input_train.shape[0], batch_size):
        labels = torch.squeeze(bin_output_train[batch:batch+batch_size].to(device))

        optimizer.zero_grad()
        outputs = latencyPred(input_train[batch:batch+batch_size].to(device))
        #print(outputs.shape)
        with torch.no_grad():
            _, predictions = torch.max(outputs.data, 1)
            predictions.to(device)
            #print("Number of predicted classes: ", len(np.unique(predictions)))
            num_correct = (predictions.to("cpu") == labels.to("cpu")).sum().item()

            batch_test.append(num_correct / outputs.shape[0] * 100)
            correct_dist_idx = (predictions == labels)

        loss = criterion(outputs, labels)
        batch_loss.append(loss.item())
        loss.backward()
        optimizer.step()

    labels = torch.squeeze(bin_output_test)
    outputs = latencyPred(input_test)
    _, predictions = torch.max(outputs.data, 1)
    predictions.to(device)
    num_correct = (predictions.to("cpu") == labels.to("cpu")).sum().item()
    test_acc = num_correct / outputs.shape[0] * 100

    loss = criterion(outputs, labels)

    epochs.append(epoch)
    losses.append(loss.item())
    tests.append(test_acc)
    
    plot_loss(epochs, losses, tests, epoch==num_epoch-1)

    print('epoch {}, loss {}'.format(epoch, losses[-1]))
    print('correct predictions {}'.format(tests[-1]))

plot_all(epochs, losses, tests, correct_dist_idx, labels)
np.save("saved/pred", predictions.cpu())
np.save("saved/label", labels.cpu())

torch.save(latencyPred.state_dict(), "saved/model.pt")

print(f"Final accuracy: {tests[-1]}%")
if do_binary_classification and tests[-1] > naive_pred:
    print(f"We do a lil bit of learning, {tests[-1] - naive_pred}")

In [38]:
bin_outputs = bin_output_test.cpu().detach().numpy()

In [28]:
output_predictions = latencyPred(input_test)
soft_predictions = scipy.special.softmax(output_predictions.cpu().detach().numpy(), axis=1)

In [46]:
np.shape(bin_outputs), np.shape(soft_predictions)

((382764, 8), (382764, 8))

In [54]:
confusion_matrix = np.zeros((class_size, class_size))
counts = np.zeros(class_size)
accuracies = np.zeros(class_size)

# Populate the confusion matrix
for t, p in zip(bin_outputs, soft_predictions):
    t_cls = np.argmax(t)
    counts[t_cls] += 1
    if (t_cls == np.argmax(p)):
        accuracies[t_cls] += 1
    for cls, prob in enumerate(p):
        confusion_matrix[t_cls, cls] += prob

for i in range(class_size):
    accuracies[i] /= counts[i]
    for j in range(class_size):
        confusion_matrix[i][j] /= counts[i]

In [51]:
print(confusion_matrix)

[[0.82926989 0.02097129 0.00666753 0.01145999 0.00676833 0.01490659
  0.05042564 0.05953074]
 [0.02393608 0.29007966 0.01790024 0.22108802 0.22057042 0.21895558
  0.00372604 0.00374396]
 [0.00634409 0.00726434 0.92736257 0.02676018 0.00202244 0.02793366
  0.0009487  0.00136401]
 [0.01061211 0.2029426  0.03700933 0.2601592  0.24147547 0.2451546
  0.00126318 0.00138351]
 [0.0061283  0.21848408 0.00810389 0.25725129 0.2655261  0.2422043
  0.00099086 0.00131119]
 [0.01468848 0.20720532 0.03903813 0.25469994 0.23626599 0.24523461
  0.00129923 0.0015683 ]
 [0.06445935 0.00415552 0.00116463 0.0016492  0.00131932 0.00186242
  0.55953392 0.36585563]
 [0.05805909 0.00313989 0.00126894 0.00133135 0.00137794 0.00165176
  0.28714384 0.64602719]]


In [55]:
accuracies

array([0.86688068, 0.34330234, 0.98765793, 0.22848416, 0.58630177,
       0.061686  , 0.41099231, 0.93374021])

In [55]:
# latencyPred = latencyPredictor(input.shape[1], 1)
class_size = len(categories)
print(class_size)
if do_binary_classification:
    class_size = 2
latencyPred = latencyPredictor(norm_input.shape[1], int(class_size))
latencyPred.double()
torch.set_num_threads(os.cpu_count())

8


In [ ]:
# criterion = torch.nn.MSELoss()
print(device)
criterion = torch.nn.CrossEntropyLoss()
latencyPred = torch.nn.DataParallel(latencyPred)
optimizer = torch.optim.SGD(latencyPred.parameters(), lr=0.01, momentum=0.961)
latencyPred.to(device)

In [31]:
# 4-feature model using 33% to test from all workloads
print(np.mean(diffs))
print(np.mean(diffs**2))
print(np.sqrt(np.mean(diffs**2)))
print(np.mean(diffs == 0))

0.6683818696238896
1.4212972736515548
1.1921817284506397
0.5209042382981705


In [27]:
latencyPred.eval()
latencyPred(torch.DoubleTensor(norm_input))

tensor([[ -6.5799,   3.9366,   4.4724,  ...,  -3.0395,  -4.5864, -14.2996],
        [-12.3283,  -2.6147,   5.0100,  ...,  -2.8761,  -4.0050, -17.3142],
        [ -6.3046,   1.4700,   4.2468,  ...,  -4.3976,  -4.5011, -16.6058],
        ...,
        [ -2.3788,  -1.9821,   2.4508,  ...,  -4.6448,  -6.0121, -19.0162],
        [-11.1532,  -7.3834,   1.0462,  ...,  -5.0560,  -5.1637, -15.9855],
        [ -2.3788,  -1.9821,   2.4508,  ...,  -4.6448,  -6.0121, -19.0162]],
       dtype=torch.float64, grad_fn=<AddmmBackward0>)

In [28]:
import pandas as pd

def layers_to_csv(layer, num):
    w_np = layer.cpu().state_dict()['weight'].numpy()
    b_np = layer.cpu().state_dict()['bias'].numpy()
    df = pd.DataFrame(w_np) #convert to a dataframe
    df.to_csv(index=False, header=False, sep=" ", path_or_buf=f"torch_model_new/linear{num}_w.csv", float_format="%015.6f") #save to file
    df = pd.DataFrame(b_np) #convert to a dataframe
    df.to_csv(index=False, header=False, sep=" ", path_or_buf=f"torch_model_new/linear{num}_b.csv", float_format="%015.6f")

In [29]:
layers_to_csv(latencyPred.module.layer_1, 0)
layers_to_csv(latencyPred.module.layer_2, 1)
layers_to_csv(latencyPred.module.layer_3, 2)
layers_to_csv(latencyPred.module.layer_out, 3)